In [2]:
import pandas as pd
import sklearn
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split


In [3]:
all_books = pd.read_csv('data/Books.csv')
all_ratings = pd.read_csv('data/Ratings.csv')
all_users = pd.read_csv('data/Users.csv')
all_books.head()

C:\Users\Ethan Rody\AppData\Local\Temp\ipykernel_9736\2654531577.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  all_books = pd.read_csv('data/Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
# check if item is a valid ISBN
def is_valid_isbn(isbn):
     if len(isbn) != 10: return False
     if not isbn.isalnum(): return False

     return True

# Converts ISBN to numerical ID
def convert_isbn_to_id(isbn, isbn_to_id):
    return isbn_to_id.get(isbn, None)

data_list = []
isbn_to_id = {}
id_counter = 0

# Converts all ISBNs in data to numerical IDs
with open('data/Ratings.csv', 'r') as file:
    next(file)
    for line in file:
        # Skips misformatted items that would cause an error
        try:
            user_id, isbn, rating = line.strip().split(',')
        except:
            continue
        
        # Skips misformatted items (not valid ISBNs)
        if not is_valid_isbn(isbn):
            continue

        # Converts ISBN to numerical ID
        if isbn not in isbn_to_id:
            isbn_to_id[isbn] = id_counter
            id_counter += 1
        
        # Builds dictionary
        item_id = isbn_to_id[isbn]

        # Add data to list of tuples
        data_list.append((user_id, item_id, float(rating)))


# Convert list to pandas data frame
df = pd.DataFrame(data_list, columns=['user_id', 'item_id', 'rating'])

# Creates inverted dictionary, use to convert numerical ID to ISBN for reporting results
id_to_isbn = {v: k for k, v in isbn_to_id.items()}

# Creates Reader to extract data
reader = Reader(line_format="user item rating", sep=',', rating_scale=(1,10))
# Loads data
data = Dataset.load_from_df(df, reader=reader)

# Splits data into train and test segments
train_data, test_data = train_test_split(data, test_size=0.25, random_state=69)


In [5]:
# Test accuracy of SVD algorithm
svd = SVD()
svd.fit(train_data)

predictions = svd.test(test_data)

accuracy.mse(predictions)

MSE: 12.1927


12.192692486000913